In [7]:
# ==========================
# 1. Mount Google Drive
# ==========================
from google.colab import drive
drive.mount('/content/drive')

# ==========================
# 2. Clone YOLOv7
# ==========================
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7

# ==========================
# 3. Install dependencies (CPU only)
# ==========================
!pip install -q torch==2.2.0 torchvision==0.17.0 torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install -q -r requirements.txt
!pip install -q kaggle matplotlib

# ==========================
# 4. Setup Kaggle API
# ==========================
import os
os.environ['KAGGLE_USERNAME'] = "shubhampr1504"   # 🔑 replace with your username
os.environ['KAGGLE_KEY'] = "10a044beb38f1a860b55bf16eb9f259b"  # 🔑 replace with your key

# ==========================
# 5. Download dataset
# ==========================
!kaggle datasets download -d niravnaik/safety-helmet-and-reflective-jacket -p ./dataset
!unzip -q ./dataset/safety-helmet-and-reflective-jacket.zip -d ./dataset/

# ==========================
# 6. Create data.yaml
# ==========================
data_yaml = """
train: dataset/safety-Helmet-Reflective-Jacket/train/images
val: dataset/safety-Helmet-Reflective-Jacket/valid/images

nc: 2
names: ['helmet', 'jacket']
"""

with open("data.yaml", "w") as f:
    f.write(data_yaml)

print("✅ data.yaml created successfully")

# ==========================
# 7. Train YOLOv7 on CPU
# ==========================
!python train.py \
    --workers 2 \
    --device cpu \
    --batch-size 4 \
    --epochs 10 \
    --img 640 640 \
    --data data.yaml \
    --weights yolov7.pt \
    --name yolov7_cpu_train \
    --cache

# ==========================
# 8. Copy trained model to Google Drive
# ==========================
!cp runs/train/yolov7_cpu_train/weights/best.pt /content/drive/MyDrive/yolov7_best_cpu.pt
print("✅ best.pt saved to Google Drive (/content/drive/MyDrive/yolov7_best_cpu.pt)")

# ==========================
# 9. Run detection
# ==========================
!python detect.py \
    --weights runs/train/yolov7_cpu_train/weights/best.pt \
    --source dataset/safety-Helmet-Reflective-Jacket/test/images \
    --device cpu \
    --name yolov7_cpu_detect

# ==========================
# 10. Visualization inside Colab
# ==========================
import glob
import random
import matplotlib.pyplot as plt
from IPython.display import Image, display

# Get detected images
detected_imgs = glob.glob("runs/detect/yolov7_cpu_detect/*.jpg")

# Show random 5 predictions
random.shuffle(detected_imgs)
for img_path in detected_imgs[:5]:
    display(Image(filename=img_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Total 1197 (delta 0), reused 0 (delta 0), pack-reused 1197 (from 1)
Receiving objects: 100% (1197/1197), 74.29 MiB | 25.03 MiB/s, done.
Resolving deltas: 100% (511/511), done.
/content/yolov7/yolov7/yolov7/yolov7/yolov7/yolov7/yolov7
Dataset URL: https://www.kaggle.com/datasets/niravnaik/safety-helmet-and-reflective-jacket
License(s): apache-2.0
 97% 502M/518M [00:01<00:00, 301MB/s]
100% 518M/518M [00:01<00:00, 397MB/s]
✅ data.yaml created successfully
YOLOR 🚀 v0.1-128-ga207844 torch 2.2.0+cpu CPU

Namespace(weights='yolov7.pt', cfg='', data='data.yaml', hyp='data/hyp.scratch.p5.yaml', epochs=10, batch_size=4, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=True, image_weights=False, device='cpu', multi